<a href="https://colab.research.google.com/github/ysforgithub/steel_defects_detection_classification/blob/master/steel_defect_images_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The Data source of this notebook is based on https://www.kaggle.com/c/severstal-steel-defect-detection/data**

In [6]:
# Get the access of the google drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Change hge working directory to the training images folder 
import os
import keras
os.chdir('/content/drive/My Drive/Colab Notebooks/kaggle/steel defect/80%_train_images')
os.getcwd()
os.listdir()

['customer_1.tfrecord',
 '80%_train_images_label_map.json',
 'Class_0',
 'Class_1',
 'Class_2',
 'Class_3',
 'Class_4',
 '80%_train_images_completePixels.csv']

In [8]:
# Print out the version of tensorflow to make sure the version is newer than 1.13. 
# A older tensorflow wouldn't have all the functions we need for this notebook 
print(tf.__version__)

1.15.0


In [9]:
# Get the training images directory and validation images directory 
train_dir = os.getcwd()
validation_dir = '/content/drive/My Drive/Colab Notebooks/kaggle/steel defect/'+'validation_images'
print("train_dir:",train_dir)
print('validation_dir:',validation_dir)

train_dir: /content/drive/My Drive/Colab Notebooks/kaggle/steel defect/80%_train_images
validation_dir: /content/drive/My Drive/Colab Notebooks/kaggle/steel defect/validation_images


# Define a CNN model to train the images classification 

In [11]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras import optimizers
from tensorflow.keras.optimizers import Adam

def create_model():

  return tf.keras.models.Sequential([
    # The input shape of the model is (256,1600,3)
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(256,1600, 3),data_format='channels_last'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(5, activation='softmax')
])

model = create_model()
model.compile(optimizer=Adam(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


*The summary of the model shows each layer of the CNN*

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 1598, 16)     448       
_________________________________________________________________
batch_normalization (BatchNo (None, 254, 1598, 16)     64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 799, 16)      0         
_________________________________________________________________
dropout (Dropout)            (None, 127, 799, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 797, 32)      4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 125, 797, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 398, 32)       0

**The image data generator is applied to preprocessing each images with some customized feature,such as rotate image, filp the image orizentally, adjust shear of the image.** 

*The purpose is to enlarge the dataset for the model training. Due to the lack of RAM in my laptop, I only use one technique of preprocessing: devide each pixel's RBG value by 255 which is the maximumn number of the pixel RGB value, to release some stress from massive model data computing.*

**Flow the image data generator through each folder of the training imanges and the validation images. Each folder represents a class of images.**

In [14]:
# Create imagedatagenerator for training images set and validation images set
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
      #rotation_range=40,
      #width_shift_range=0.2,
      #height_shift_range=0.2,
      #shear_range=0.2,
      #zoom_range=0.2,
      #horizontal_flip=True,
      #fill_mode='nearest'
                                  )
test_datagen = ImageDataGenerator(rescale=1./255,
      #rotation_range=40,
      #width_shift_range=0.2,
      #height_shift_range=0.2,
      #shear_range=0.2,
      #zoom_range=0.2,
      #horizontal_flip=True,
      #fill_mode='nearest'
                                 )
# flow training images in batches of 32 using train_datagen generator 
train_generator = train_datagen.flow_from_directory(
  train_dir,
  target_size = (256,1600),
  color_mode='rgb',
  batch_size = 32,
  class_mode = 'sparse'
)

# flow validation images in batches 
validation_generator = test_datagen.flow_from_directory(
  validation_dir,
  target_size=(256,1600),
  color_mode='rgb',
  batch_size = 32,
  class_mode = 'sparse'
)

Found 10397 images belonging to 5 classes.
Found 2600 images belonging to 5 classes.


Define the callback function used during the model training:


1.   callback 1: earlystop, this function will stop the model training when the model detect there is no accracy improment between the last two training epoch. The setting of the accuracy difference here is 0.05
2.   callback 2: modelcheckpoint, this function will check the accuracy after each epoch of training. Then save the best model's weights to a H5 file based on the loss value of each training. 
3.   callback 3: tensorboard, this function will log the accuracy, loss info of each epoch then save it as a file. After the training is completed or stopped, launch the tensorboard will be able to visualize the training accuracy and loss curve. 
4.   callback 4: learning scheduler, this function will adjust the hyperparameter of the training model, lr(learning rate). At the begining of the model training, the learning rate is bigger which means the model's parameters(weigths) will be adjusted more aggresively. Then the learning rate is decreased when the training is getting closer to the end to prevent the model is overfitted. 



In [0]:
# keras callbacks
import keras, os, datetime
from keras.callbacks import ModelCheckpoint
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
k_earlystop = keras.callbacks.EarlyStopping(monitor='acc', min_delta=0.05, patience=1, verbose=0, mode='auto', baselinestore_best_weights=False)

k_checkpoint = keras.callbacks.ModelCheckpoint('/content/drive/My Drive/Colab Notebooks/kaggle/steel defect/weights_sets/'+current_time+'_CNN.hdf5', monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

log_directory="/content/drive/My Drive/Colab Notebooks/kaggle/steel defect/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
k_tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_directory,histogram_freq=0, batch_size=32, write_graph=True, update_freq='epoch')

def scheduler(epoch,lr):
  if epoch < 10:
    pass
  else:
    lr = 0.001 * tf.math.exp(0.1 * (10 - epoch))
  return lr
k_learningrate = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=1)

In [0]:
'''
from keras.models import load_model
model.load_weights('/content/drive/My Drive/Colab Notebooks/kaggle/steel defect/tmp_10_08.h5')

In [19]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=10397//32,  # 2000 images = batch_size * steps
      epochs=20,
      validation_data=validation_generator,
      validation_steps=2600//32,  # 1000 images = batch_size * steps
      callbacks=[k_tensorboard,k_earlystop,k_checkpoint,k_learningrate]
)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 1/20
 81/324 [======>.......................] - ETA: 39:18 - loss: 1.7646 - acc: 0.4742
Epoch 00001: loss improved from inf to 2.16486, saving model to /content/drive/My Drive/Colab Notebooks/kaggle/steel defect/weights_sets/20200211-044903_CNN.hdf5
324/324 [==============================] - 8278s 26s/step - loss: 2.1646 - acc: 0.4233 - val_loss: 1.7646 - val_acc: 0.4742

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 2/20
 81/324 [======>.......................] - ETA: 25:13 - loss: 1.2539 - acc: 0.4742
Epoch 00002: loss improved from 2.16486 to 1.36739, saving model to /content/drive/My Drive/Colab Notebooks/kaggle/steel defect/weights_sets/20200211-044903_CNN.hdf5
324/324 [==============================] - 8026s 25s/step - loss: 1.3674 - acc: 0.4450 - val_loss: 1.2539 - val_acc: 0.4742


In [0]:
# model.save('/content/drive/My Drive/Colab Notebooks/kaggle/steel defect//weights_sets/CNN_200_08_v1.hdf5')

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.figure()


In [0]:
%tensorboard --logdir logs/fit